In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from biometric_helper import *
from numpy.random import shuffle, choice
%matplotlib inline

In [2]:
accel_data = clean_data(pd.read_csv("ProjectData.csv"))
accel_data.A_tot /= accel_data.A_tot.max()
accel_data.A_tot -= accel_data.A_tot.mean()

biometric_helper.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  group["Samplerate"] = 50


In [4]:
# accel_data.describe()
batch_size=100
sample_time = 3

In [5]:
# def train_test_split(feats, labels, ratio=.7):
    

class DataLoader(object):
    def __init__(self, data, rate, b_size=100, time=3):
        train_samples = []
        train_labels = []
        test_samples = []
        test_labels = []
        
        seconds = 3
        chunksize = int(rate*seconds)
        self.chunksize = chunksize
        
        # Split the data into 3 second segments
        grouped = data.groupby("JointID")
        for person in grouped.groups:
            temp = grouped.get_group(person)
            n_samples = temp.shape[0]//chunksize
            feats = temp.loc[:,"A_tot"].as_matrix()
            labels = np.ones(n_samples)*temp.JointID.as_matrix()[0]
            samples = np.array([feats[chunksize*n:chunksize*(n+1)].flatten()\
                                for n in xrange(n_samples)])
            
            test_rows = choice(n_samples, size=np.floor(.3*n_samples), replace=False)
            test_samples.append(samples[test_rows])
            test_labels.append(labels[test_rows])
            train_mask = np.ones(n_samples, dtype=bool)
            train_mask[test_rows] = 0
            train_samples.append(samples[train_mask])
            train_labels.append(labels[train_mask])
        
        self.train_samples = np.vstack(tuple(train_samples))
        self.test_samples = np.vstack(tuple(test_samples))
        
        self.train_size = self.train_samples.shape[0]
        self.test_size = self.test_samples.shape[0]
        
        self.train_labels = np.hstack(tuple(train_labels)).flatten().reshape((self.train_size,1))
        self.train_labels = pd.get_dummies(pd.DataFrame(self.train_labels), columns=[0]).as_matrix()

        self.test_labels = np.hstack(tuple(test_labels)).flatten().reshape((self.test_size,1))
        self.test_labels = pd.get_dummies(pd.DataFrame(self.test_labels), columns=[0]).as_matrix()
        
        self.train_iter = 0
        self.test_iter = 0
        
        self.b_size = b_size
        self.n_train_batches = self.train_size//self.b_size
        self.n_test_batches = self.test_size//self.b_size
        
        
    def reset_batches(self):
        merged_train = np.hstack((self.train_samples, self.train_labels))
        shuffle(merged_train)
        merged_test = np.hstack((self.test_samples, self.test_labels))
        shuffle(merged_test)
        
        self.train_iter = 0
        self.test_iter = 0
        
        self.train_samples = merged_train[:,:self.chunksize]
        self.train_labels = merged_train[:,self.chunksize:]
        self.test_samples = merged_test[:,:self.chunksize]
        self.test_labels = merged_test[:,self.chunksize:]
#         print self.train_samples.shape
#         print self.train_labels.shape
#         print self.train_labels[:5]
        
    def train_batch(self):
        a = self.train_iter*self.b_size
        b = (self.train_iter+1)*self.b_size
        self.train_iter += 1
        return self.train_samples[a:b], self.train_labels[a:b] 
    
    
    def test_batch(self):
        a = self.test_iter*self.b_size
        b = (self.test_iter+1)*self.b_size
        self.test_iter += 1
        return self.test_samples[a:b], self.test_labels[a:b]
    
    def num_train_batches(self):
        return self.n_train_batches
    
    def num_test_batches(self):
        return self.n_test_batches
    
        
loader = DataLoader(accel_data,50, b_size=batch_size, time=sample_time)
loader.reset_batches()
j, k = loader.test_batch()
print j.shape
print k.shape

(100, 150)
(100, 42)


/usr/local/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [16]:
def conv1d( in_var, output_dim, width=10, stride=1, name="conv1d" ):
    k_x = width  # filter width
    d_x = stride  # x stride

    with tf.variable_scope( name ):
        W = tf.get_variable( "W", [k_x, in_var.get_shape()[-1], output_dim],
                             initializer=tf.truncated_normal_initializer(stddev=0.02) )
        b = tf.get_variable( "b", [output_dim], initializer=tf.constant_initializer(0.01) )

        conv = tf.nn.conv1d( in_var, W, stride=stride, padding='SAME' )
        conv = tf.reshape( tf.nn.bias_add( conv, b ), conv.get_shape() )

        return conv

    
def linear( in_var, output_size, name="linear", stddev=0.02, bias_val=.01 ):
    shape = in_var.get_shape().as_list()

    with tf.variable_scope( name ):
        W = tf.get_variable( "W", [shape[1], output_size], tf.float32,
                              tf.random_normal_initializer( stddev=stddev ) )
        b = tf.get_variable( "b", [output_size],
                             initializer=tf.constant_initializer( bias_val ))

        return tf.matmul( in_var, W ) + b    
    
    
    
def conv_resnet(in_var, out_shape, stride=1, width=5, name='resnet'):
    """
    Make a ResNet layer, i.e. two convolutions and an identity mapping
    """
    x_stride = stride
    std = .02
    const = .01
    
    if stride!=1 and stride!=2:
        print stride
        raise ValueError("Stride must be 1 or 2")
    
    with tf.variable_scope(name):
        
        # First convolution in ResNet layer
        conv1 = conv1d(in_var, out_shape, stride=stride, width=width)
        
        # Relu-fy the convolutional layer
        relu_layer = tf.nn.relu(conv1)
        
        # Second convolution in ResNet layer
        conv2 = conv1d(in_var, out_shape, width=width)
        
        if stride==1:
            ident= in_var
#         else:
#             w3 = tf.get_variable("W3", [1, 1, in_var.get_shape()[-1], out_shape], 
#                              initializer=tf.truncated_normal_initializer(stddev=std))
#             in_var = tf.nn.conv1d(in_var, w3, stride=1 padding="SAME")
#             in_var = tf.reshape(in_var)
#             ident = tf.nn.max_pool(in_var, ksize=[1,2, 1, 1],
#                         strides=[1, 2, 2, 1], padding='SAME')
# #             print in_var.get_shape()
        
#         # Add identity and run through another relu
        add_x = tf.nn.relu(conv2 + ident)
    return add_x
        
        
# def siamese_block(imgs):
#     imgs = tf.reshape(imgs, [batch_size, x_dim, x_dim, 1])
#     imgs = tf.nn.avg_pool(imgs,[1,2,2,1], strides=[1,2,2,1], padding='SAME')
#     conv0 = conv2d(imgs, 64, name='first_conv')
#     res1 = resnet_block(conv0, 64, name='res1')
#     res2 = resnet_block(res1, 64, name='res2')
#     res3 = resnet_block(res2, 128, stride=2, name='res3')
#     res4 = resnet_block(res3, 128, name='res4')
#     res5 = resnet_block(res4, 256, stride=2, name='res5')
#     pool1 = tf.reshape(tf.nn.max_pool(res5, [1,2,2,1], strides=[1,2,2,1], padding='SAME'), [batch_size,-1])
#     fc_1 = linear(pool1, 1024, name='fc1')
#     return fc_1

## First Architecture--Convnet experiment

In [19]:
tf.reset_default_graph()
sess = tf.Session()

x = tf.placeholder(tf.float32, [batch_size, 50*sample_time], name="x")
y = tf.placeholder(tf.float32, [batch_size, 42], name="y")


with tf.name_scope("FC_1"):
    fc1 = tf.reshape(linear(x, 256, name="fc_1"), [batch_size, 256, 1])
    relu1 = tf.nn.relu(fc1)
    
with tf.name_scope("Conv_2"):
    conv2 = conv1d(relu1, 64, name='conv2')
    relu2 = tf.nn.relu(conv2)
    
with tf.name_scope("FC_3"):
    fc4 = linear(tf.reshape(relu2, [batch_size, -1]), 1024, name='fc_3')
    relu4 = tf.nn.relu(fc4)

with tf.name_scope("Score"):
    fc5 = linear(relu4, 42, name='fc_4')
    score = tf.log(tf.nn.softmax(fc5))

with tf.name_scope("Loss"):
    # l2_regularization = tf.reduce_sum([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
    loss = tf.reduce_mean(-tf.reduce_sum(score * y)) #+ .01*l2_regularization

with tf.name_scope("Accuracy"):
    correct_prediction = tf.equal(tf.argmax(score,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


with tf.name_scope("Optim"):
    optim = tf.train.AdamOptimizer(1e-5).minimize(loss)



sess.run(tf.initialize_all_variables())

In [20]:
loader = DataLoader(accel_data,50, b_size=batch_size, time=sample_time)

num_train = loader.num_train_batches()
num_test = loader.num_test_batches()
loss_vals = []
train_acc = []
summary_writer = tf.train.SummaryWriter( "./final_logs", graph=sess.graph )
for i in xrange(1,2):
    loader.reset_batches()
    step_loss = []
    
    temp_acc = []
    for j in xrange(num_train):
        xs, ys = loader.train_batch()
        l = sess.run(loss, feed_dict={x:xs, y:ys})
        acc = sess.run(accuracy, feed_dict={x:xs, y:ys})
        sess.run(optim, feed_dict={x:xs, y:ys})
        
        step_loss.append(l)
        temp_acc.append(acc)
    loss_vals.append(np.mean(step_loss))
    train_acc.append(np.mean(temp_acc))
    print i, np.mean(loss_vals), np.mean(temp_acc)
        
    
    if i%10==0:
        acc_scores = []
        for k in xrange(num_test):
            loader.reset_batches()
            xs, ys = loader.test_batch()
            acc = sess.run(accuracy, feed_dict={x:xs, y:ys})
            acc_scores.append(acc)
        test_acc = np.mean(acc_scores)
        print i, "Test Accuracy:", test_acc
        print sess.run(fc5, feed_dict={x:xs, y:ys})
        
summary_writer.close()
    

/usr/local/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


1 360.28 0.418837


## Second Architecture: Fully Connected Net

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [batch_size, 50*sample_time])
y = tf.placeholder(tf.float32, [batch_size, 42])

fc1 = tf.reshape(linear(x, 600, name="fc_1"), [batch_size, -1])
relu1 = tf.nn.relu(fc1)

fc4 = linear(tf.reshape(relu1, [batch_size, -1]), 600, name='fc_4')
drop = tf.nn.dropout(fc4, .5)

fc5 = linear(drop, 42, name='fc_5')

l2_regularization = tf.reduce_mean([tf.nn.l2_loss(var) for var in tf.trainable_variables()])

score = tf.log(tf.nn.softmax(fc5))
loss = tf.reduce_mean(-tf.reduce_sum(score * y) + 1e-5*l2_regularization)

correct_prediction = tf.equal(tf.argmax(score,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



optim = tf.train.AdamOptimizer(1e-5).minimize(loss)



sess.run(tf.initialize_all_variables())

In [ ]:
sess.run(fc1, feed_dict={x:xs, y:ys})

In [ ]:
# Load data and train network
loader = DataLoader(accel_data,50, b_size=batch_size, time=sample_time)

num_train = loader.num_train_batches()
num_test = loader.num_test_batches()
loss_vals = []
for i in xrange(1000):
    loader.reset_batches()
    step_loss = []
    for j in xrange(num_train):
        xs, ys = loader.train_batch()
        sess.run(optim, feed_dict={x:xs, y:ys})
        l = sess.run(loss, feed_dict={x:xs, y:ys})
        step_loss.append(l)
    loss_vals.append(np.mean(step_loss))
    print i, loss_vals[-1]
        
    
    if i%10==0:
        acc_scores = []
        for k in xrange(num_test):
            loader.reset_batches()
            xs, ys = loader.test_batch()
            acc = sess.run(accuracy, feed_dict={x:xs, y:ys})
            acc_scores.append(acc)
        test_acc = np.mean(acc_scores)
        print i, "Test Accuracy:", test_acc

In [ ]:
loader = DataLoader(accel_data,50, b_size=batch_size, time=sample_time)

num_train = loader.num_train_batches()
num_test = loader.num_test_batches()
loss_vals = []
# for i in xrange(100):
loader.reset_batches()
step_loss = []
for j in xrange(num_train):
    xs, ys = loader.train_batch()
#         sess.run(optim, feed_dict={x:xs, y:ys})
    l = sess.run(score, feed_dict={x:xs, y:ys})
    print l
    step_loss.append(l)
loss_vals.append(np.mean(step_loss))
print i, loss_vals[-1]